Import Pandas package so we can use Pandas dataframes and json package so we can use json files

In [799]:
import pandas as pd
import json

Function to load the .csv files containing the static tables. This function takes a list as input where each list item is the string of a pathway to a .csv file. The files are then opened as Pandas dataframes and stored in a dictionary where the filename is the key and the dataframe is the value.

In [800]:
def Load_csvs(list):
    dic_of_dfs = {}
    for i in list:
        dfname = i.rsplit('\\', 1)[-1]
        dic_of_dfs[dfname] = pd.read_csv(i)
    return dic_of_dfs

In [801]:
list_of_pathways_to_csvs = ['C:\\Users\\josep\\OneDrive\\Documents\\GitHub\\multipeattools\\SET_Tool\\SET_Tool\\csv_files\\C_Content_Soil_Static_Values.csv',
     'C:\\Users\\josep\\OneDrive\\Documents\\GitHub\\multipeattools\\SET_Tool\\SET_Tool\\csv_files\\CO2_Equivalents.csv',
     'C:\\Users\\josep\\OneDrive\\Documents\\GitHub\\multipeattools\\SET_Tool\\SET_Tool\\csv_files\\Crop_Residue_Paludi.csv',
     'C:\\Users\\josep\\OneDrive\\Documents\\GitHub\\multipeattools\\SET_Tool\\SET_Tool\\csv_files\\Fossil_Energy_Static_Values.csv',
     'C:\\Users\\josep\\OneDrive\\Documents\\GitHub\\multipeattools\\SET_Tool\\SET_Tool\\csv_files\\GEST_2_Static_Values.csv',
     'C:\\Users\\josep\\OneDrive\\Documents\\GitHub\\multipeattools\\SET_Tool\\SET_Tool\\csv_files\\Soil_Moisture_Class_Static_Values.csv']

Function to create a json containing the user input values. The input values in this function are merely standins until actual user input is provided, at which point the json file will be updated.

In [802]:
def Create_Input_Json():
    inputs = {'General Site Data': {'Site Name': 'Dutch Example',
                                     'Total Area': 1, 
                                     'Coordinates': [0, 0], 
                                     'Elevation': 0, 
                                     'Peat Type': 'Sphagnum',
                                     'Peat Thickness': 100,
                                     'Year Rewetting Started': 2022}, 
            'Baseline': {'Median Groundwater Level in Summer': -10,
                         'Vegetation Class': 'G5s:Wet Grassland with Shunt Species',
                         'Animal Manure Applied': 150,
                         'Organic Fertilizer Applied': 10,
                         'Synthetic Fertilizer': 'Nitrate Based',
                         'Amount Applied': 20,
                         'Grazing Animals': 'No Animal',
                         'Average Number of Grazing Animals': 0,
                         'Average Number of Grazing Days per Year': 0,
                         'Crop Yield': 12,
                         'Crop Residue': 'No',
                         'Crop': 'Cattail (Typha Sp.)',
                         'Litres of Diesel per Site': 0,
                         'Electricity per Site': 0},
            'Rewetting': {'Median Groundwater Level in Summer': -55,
                         'Vegetation Class': 'G2:Moist Grassland',
                         'Animal Manure Applied': 100,
                         'Organic Fertilizer Applied': 50,
                         'Synthetic Fertilizer': 'Nitrate Based',
                         'Amount Applied': 50,
                         'Grazing Animals': 'Sheep',
                         'Average Number of Grazing Animals': 10,
                         'Average Number of Grazing Days per Year': 100,
                         'Crop Yield': 24,
                         'Crop Residue': 'Yes',
                         'Crop': 'Alder (Alnus Sp.)',
                         'Litres of Diesel per Site': 10,
                         'Electricity per Site': 10,
                         'Crop Use': 'Paper'}}
    
    with open('user_input_SET.json', 'w') as outfile:
        print(json.dumps(inputs, indent = 5), file = outfile)

In [803]:
Create_Input_Json()

Create the Data tab by calling information from the Input json and GEST 2.0 dataframe. This function takes two parameters. The first is a string of the name of the json file where the user input data is stored, the second is the dataframe which contains the GEST 2.0 table. This function returns a pandas dataframe which contains the output provided in the data tab of the excel file.

In [804]:
def Create_Data_Tab(user_input, gest):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initiate the Data dict
    data = {'Baseline': {}, 'Rewetting': {}}

    #Start populating the data dict with simple data
    data['Baseline']['Vegetation Class'] = user_input['Baseline']['Vegetation Class']
    data['Baseline']['Grazing Animals'] = user_input['Baseline']['Grazing Animals']
    data['Baseline']['Synthetic Fertilizer'] = user_input['Baseline']['Synthetic Fertilizer']

    data['Rewetting']['Vegetation Class'] = user_input['Rewetting']['Vegetation Class']
    data['Rewetting']['Grazing Animals'] = user_input['Rewetting']['Grazing Animals']
    data['Rewetting']['Synthetic Fertilizer'] = user_input['Rewetting']['Synthetic Fertilizer']

    #Calculate the animal emission factor and nitrogen excretion per head per year
    if data['Baseline']['Grazing Animals'] == 'No Animals':
        data['Baseline']['Animal Emission Factor'] = 0
        data['Baseline']['Nitrogen Excretion'] = 0
    elif data['Baseline']['Grazing Animals'] == 'Dairy Cattle':
        data['Baseline']['Animal Emission Factor'] = 0.02
        data['Baseline']['Nitrogen Excretion'] = 130.1
    elif data['Baseline']['Grazing Animals'] == 'Beef Cattle':
        data['Baseline']['Animal Emission Factor'] = 0.02
        data['Baseline']['Nitrogen Excretion'] = 28.2
    elif data['Baseline']['Grazing Animals'] == 'Sheep':
        data['Baseline']['Animal Emission Factor'] = 0.01
        data['Baseline']['Nitrogen Excretion'] = 9.9
    elif data['Baseline']['Grazing Animals'] == 'Goats':
        data['Baseline']['Animal Emission Factor'] = 0.01
        data['Baseline']['Nitrogen Excretion'] = 7.4
    elif data['Baseline']['Grazing Animals'] == 'Water Buffalo':
        data['Baseline']['Animal Emission Factor'] = 0.02
        data['Baseline']['Nitrogen Excretion'] = 76.5

    if data['Rewetting']['Grazing Animals'] == 'No Animals':
        data['Rewetting']['Animal Emission Factor'] = 0
        data['Rewetting']['Nitrogen Excretion'] = 0
    elif data['Rewetting']['Grazing Animals'] == 'Dairy Cattle':
        data['Rewetting']['Animal Emission Factor'] = 0.02
        data['Rewetting']['Nitrogen Excretion'] = 130.1
    elif data['Rewetting']['Grazing Animals'] == 'Beef Cattle':
        data['Rewetting']['Animal Emission Factor'] = 0.02
        data['Rewetting']['Nitrogen Excretion'] = 28.2
    elif data['Rewetting']['Grazing Animals'] == 'Sheep':
        data['Rewetting']['Animal Emission Factor'] = 0.01
        data['Rewetting']['Nitrogen Excretion'] = 9.9
    elif data['Rewetting']['Grazing Animals'] == 'Goats':
        data['Rewetting']['Animal Emission Factor'] = 0.01
        data['Rewetting']['Nitrogen Excretion'] = 7.4
    elif data['Rewetting']['Grazing Animals'] == 'Water Buffalo':
        data['Rewetting']['Animal Emission Factor'] = 0.02
        data['Rewetting']['Nitrogen Excretion'] = 76.5

    #Calculate synthetic fertilizer emission factor
    if data['Baseline']['Synthetic Fertilizer'] == 'Nitrate Based':
        data['Baseline']['Fertilizer Emission Factor'] = 0.015
    elif data['Baseline']['Synthetic Fertilizer'] == 'Ammonium Based':
        data['Baseline']['Fertilizer Emission Factor'] = 0.01

    if data['Rewetting']['Synthetic Fertilizer'] == 'Nitrate Based':
        data['Rewetting']['Fertilizer Emission Factor'] = 0.015
    elif data['Rewetting']['Synthetic Fertilizer'] == 'Ammonium Based':
        data['Rewetting']['Fertilizer Emission Factor'] = 0.01

    #Find Crop Residue number
    if user_input['Baseline']['Crop Residue'] == 'Yes':
        data['Baseline']['Crop Residue'] = 1
    else:
        data['Baseline']['Crop Residue'] = 2

    if user_input['Rewetting']['Crop Residue'] == 'Yes':
        data['Rewetting']['Crop Residue'] = 1
    else:
        data['Rewetting']['Crop Residue'] = 2

    #Find Number of Crop Name
    if user_input['Baseline']['Crop'] == 'Cattail (Typha Sp.)':
        data['Baseline']['Crop'] = 1
    elif user_input['Baseline']['Crop'] == 'Reed (Phragmites Australis)':
        data['Baseline']['Crop'] = 2
    elif user_input['Baseline']['Crop'] == 'Peat Moss (Sphagnum Sp.) ':
        data['Baseline']['Crop'] = 3
    elif user_input['Baseline']['Crop'] == 'Grasses like Reed Canary Grass':
        data['Baseline']['Crop'] = 4
    elif user_input['Baseline']['Crop'] == 'Alder (Alnus Sp.)':
        data['Baseline']['Crop'] = 5
    elif user_input['Baseline']['Crop'] == 'Other':
        data['Baseline']['Crop'] = 6

    if user_input['Rewetting']['Crop'] == 'Cattail (Typha Sp.)':
        data['Rewetting']['Crop'] = 1
    elif user_input['Rewetting']['Crop'] == 'Reed (Phragmites Australis)':
        data['Rewetting']['Crop'] = 2
    elif user_input['Rewetting']['Crop'] == 'Peat Moss (Sphagnum Sp.) ':
        data['Rewetting']['Crop'] = 3
    elif user_input['Rewetting']['Crop'] == 'Grasses like Reed Canary Grass':
        data['Rewetting']['Crop'] = 4
    elif user_input['Rewetting']['Crop'] == 'Alder (Alnus Sp.)':
        data['Rewetting']['Crop'] = 5
    elif user_input['Rewetting']['Crop'] == 'Other':
        data['Rewetting']['Crop'] = 6

    #Find Product Use Number
    if user_input['Rewetting']['Crop Use'] == 'Building Materials e.g insulation, taching, timber':
        data['Rewetting']['Crop Use'] = 1
    elif user_input['Rewetting']['Crop Use'] == 'Bedding Material':
        data['Rewetting']['Crop Use'] = 2
    elif user_input['Rewetting']['Crop Use'] == 'Food Application':
        data['Rewetting']['Crop Use'] = 3
    elif user_input['Rewetting']['Crop Use'] == 'Fodder/Feed Application':
        data['Rewetting']['Crop Use'] = 4
    elif user_input['Rewetting']['Crop Use'] == 'Energy Use: biogas, combustion, wood etc':
        data['Rewetting']['Crop Use'] = 5
    elif user_input['Rewetting']['Crop Use'] == 'Paper':
        data['Rewetting']['Crop Use'] = 6
    elif user_input['Rewetting']['Crop Use'] == 'Extraction of Ingredients/Building Blocks: proteins, fibres, cellulose etc':
        data['Rewetting']['Crop Use'] = 7
    elif user_input['Rewetting']['Crop Use'] == 'High Quality Substrate in Horticulture':
        data['Rewetting']['Crop Use'] = 8
    elif user_input['Rewetting']['Crop Use'] == 'Other Uses/Unknown':
        data['Rewetting']['Crop Use'] = 9

    #Perform vegetation class emission calculations
    veg_num_base = data['Baseline']['Vegetation Class'].split(':', 1)[0]
    veg_num_rewet = data['Rewetting']['Vegetation Class'].split(':', 1)[0]
    
    for i in range(2, 44):
        if veg_num_base == gest.iloc[i]['Name']:
            data['Baseline']['Vegetation CH4 GWP'] = gest.iloc[i]['CH4']
            data['Baseline']['Vegetation CO2 GWP'] = gest.iloc[i]['CO2']
            data['Baseline']['Vegetation Total C-Flux GWP'] = gest.iloc[i]['Total C-flux (GWP)']
        if veg_num_rewet == gest.iloc[i]['Name']:
            data['Rewetting']['Vegetation CH4 GWP'] = gest.iloc[i]['CH4']
            data['Rewetting']['Vegetation CO2 GWP'] = gest.iloc[i]['CO2']
            data['Rewetting']['Vegetation Total C-Flux GWP'] = gest.iloc[i]['Total C-flux (GWP)']

    return pd.DataFrame.from_dict(data)
    


In [805]:
dict_of_csvs = Load_csvs(list_of_pathways_to_csvs)
gest = dict_of_csvs['GEST_2_Static_Values.csv']

In [806]:
data_tab = Create_Data_Tab('user_input_SET.json', gest)
data_tab

,Baseline,Rewetting
Vegetation Class,G5s:Wet Grassland with Shunt Species,G2:Moist Grassland
Grazing Animals,No Animal,Sheep
Synthetic Fertilizer,Nitrate Based,Nitrate Based
Fertilizer Emission Factor,0.015,0.015
Crop Residue,2,1
Crop,1,5
Vegetation CH4 GWP,2.93,0.01
Vegetation CO2 GWP,-3.89,19.37
Vegetation Total C-Flux GWP,-1,19.5
Animal Emission Factor,NaN,0.01


Create the Data tab by calling information from the Input json and the Data Tab dataframe. This function takes two parameters. The first is a string of the name of the json file where the user input data is stored, the second is the dataframe which contains the Data Tab dataframe. This function returns a pandas dataframe which contains the output provided in the data tab of the excel file.

In [807]:
def Create_Crop_Use_Tab(user_input, data):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialize Crop Use dictionary
    crop_use = {}

    #Populate the Crop Use dictionary
    crop_use['Crop Yield'] = user_input['Rewetting']['Crop Yield']

    if data.loc['Crop Use']['Rewetting'] == 2 or data.loc['Crop Use']['Rewetting'] == 3 or data.loc['Crop Use']['Rewetting'] == 4 or data.loc['Crop Use']['Rewetting'] == 9:
        crop_use['Product Weight'] = 0
    else:
        crop_use['Product Weight'] = crop_use['Crop Yield']

    crop_use['Carbon Weight'] = crop_use['Product Weight']*0.475
    crop_use['Ton CO2'] = crop_use['Carbon Weight']*(44/12)
    crop_use['Ton CO2 per Ha'] = crop_use['Ton CO2']*(-1)
    crop_use['Ton CO2 per Site'] = crop_use['Ton CO2 per Ha']*user_input['General Site Data']['Total Area']

    #Save Crop Use as a pandas dataframe
    crop_use = {'Values': crop_use}
    return pd.DataFrame.from_dict(crop_use)

In [808]:
crop_use_tab = Create_Crop_Use_Tab('user_input_SET.json', data_tab)
crop_use_tab

,Values
Carbon Weight,11.4
Crop Yield,24.0
Product Weight,24.0
Ton CO2,41.8
Ton CO2 per Ha,-41.8
Ton CO2 per Site,-41.8


Create the C Content Soil Tab by using information from the User Input json.

In [809]:
def Create_C_Content_Soil_Tab(user_input):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialize the C Content Soil dictionary
    c_content = {}

    #Populate C Content dictionary
    c_content['Peat Type'] = user_input['General Site Data']['Peat Type']

    if c_content['Peat Type'] == 'Sphagnum':
        c_content['C Content kg/C/Ha/cm thickness'] = 0.3496
    elif c_content['Peat Type'] == 'Herbaceous':
        c_content['C Content kg/C/Ha/cm thickness'] = 0.5959
    elif c_content['Peat Type'] == 'Woody':
        c_content['C Content kg/C/Ha/cm thickness'] = 0.54972
    elif c_content['Peat Type'] == 'Brown moss':
        c_content['C Content kg/C/Ha/cm thickness'] = 0.84783
    elif c_content['Peat Type'] == 'Unknown':
        c_content['C Content kg/C/Ha/cm thickness'] = 0.55224
    elif c_content['Peat Type'] == 'Humified':
        c_content['C Content kg/C/Ha/cm thickness'] = 0.91008

    c_content['Peat Thickness'] = user_input['General Site Data']['Peat Thickness']
    c_content['C-stock (Ton C per Ha)'] = c_content['Peat Thickness']*c_content['C Content kg/C/Ha/cm thickness']*10
    c_content['C-stock (tCO2-eq per Ha)'] = c_content['C-stock (Ton C per Ha)']*(44/12)

    #Save C Content Soil tab as a pandas dataframe
    c_content = {'Values': c_content}
    return pd.DataFrame.from_dict(c_content)

In [810]:
c_content_tab = Create_C_Content_Soil_Tab('user_input_SET.json')
c_content_tab

,Values
C Content kg/C/Ha/cm thickness,0.3496
C-stock (Ton C per Ha),349.6
C-stock (tCO2-eq per Ha),1281.866667
Peat Thickness,100
Peat Type,Sphagnum


Create the Soil Moisture Classes Tab by using information from the User Input json.

In [811]:
def Create_Soil_Moisture_Classes_Tab(user_input):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialize the Soil Moisture Classes dictionary
    sm_classes = {'Baseline': {}, 'Rewetting': {}}

    #Populate dictionary
    if user_input['Baseline']['Median Groundwater Level in Summer'] >= 0 and user_input['Baseline']['Median Groundwater Level in Summer'] <= 140:
        sm_classes['Baseline']['Summer Moisture Class Number'] = '6+'
        sm_classes['Baseline']['Summer Moisture Class Name'] = 'Flooded (lower eulittoral)'
        sm_classes['Baseline']['Summer Moisture Class'] = '6+ (Flooded (lower eulittoral))'
    elif user_input['Baseline']['Median Groundwater Level in Summer'] >= -10 and user_input['Baseline']['Median Groundwater Level in Summer'] < 0:
        sm_classes['Baseline']['Summer Moisture Class Number'] = '5+'
        sm_classes['Baseline']['Summer Moisture Class Name'] = 'Wet (upper eulittoral)'
        sm_classes['Baseline']['Summer Moisture Class'] = '5+ (Wet (upper eulittoral))'
    elif user_input['Baseline']['Median Groundwater Level in Summer'] >= -20 and user_input['Baseline']['Median Groundwater Level in Summer'] < -10:
        sm_classes['Baseline']['Summer Moisture Class Number'] = '4+'
        sm_classes['Baseline']['Summer Moisture Class Name'] = 'Semi wet (very moist)'
        sm_classes['Baseline']['Summer Moisture Class'] = '4+ (Semi wet (very moist))'
    elif user_input['Baseline']['Median Groundwater Level in Summer'] >= -45 and user_input['Baseline']['Median Groundwater Level in Summer'] < -20:
        sm_classes['Baseline']['Summer Moisture Class Number'] = '3+'
        sm_classes['Baseline']['Summer Moisture Class Name'] = 'Moist'
        sm_classes['Baseline']['Summer Moisture Class'] = '3+ (Moist)'
    elif user_input['Baseline']['Median Groundwater Level in Summer'] >= -85 and user_input['Baseline']['Median Groundwater Level in Summer'] < -45:
        sm_classes['Baseline']['Summer Moisture Class Number'] = '2+'
        sm_classes['Baseline']['Summer Moisture Class Name'] = 'Moderate moist'
        sm_classes['Baseline']['Summer Moisture Class'] = '2+ (Moderate moist)'
    elif user_input['Baseline']['Median Groundwater Level in Summer'] < -85:
        sm_classes['Baseline']['Summer Moisture Class Number'] = '2-'
        sm_classes['Baseline']['Summer Moisture Class Name'] = 'Moderate dry'
        sm_classes['Baseline']['Summer Moisture Class'] = '2- (Moderate dry)'

    if user_input['Rewetting']['Median Groundwater Level in Summer'] >= 0 and user_input['Rewetting']['Median Groundwater Level in Summer'] <= 140:
        sm_classes['Rewetting']['Summer Moisture Class Number'] = '6+'
        sm_classes['Rewetting']['Summer Moisture Class Name'] = 'Flooded (lower eulittoral)'
        sm_classes['Rewetting']['Summer Moisture Class'] = '6+ (Flooded (lower eulittoral))'
    elif user_input['Rewetting']['Median Groundwater Level in Summer'] >= -10 and user_input['Rewetting']['Median Groundwater Level in Summer'] < 0:
        sm_classes['Rewetting']['Summer Moisture Class Number'] = '5+'
        sm_classes['Rewetting']['Summer Moisture Class Name'] = 'Wet (upper eulittoral)'
        sm_classes['Rewetting']['Summer Moisture Class'] = '5+ (Wet (upper eulittoral))'
    elif user_input['Rewetting']['Median Groundwater Level in Summer'] >= -20 and user_input['Rewetting']['Median Groundwater Level in Summer'] < -10:
        sm_classes['Rewetting']['Summer Moisture Class Number'] = '4+'
        sm_classes['Rewetting']['Summer Moisture Class Name'] = 'Semi wet (very moist)'
        sm_classes['Rewetting']['Summer Moisture Class'] = '4+ (Semi wet (very moist))'
    elif user_input['Rewetting']['Median Groundwater Level in Summer'] >= -45 and user_input['Rewetting']['Median Groundwater Level in Summer'] < -20:
        sm_classes['Rewetting']['Summer Moisture Class Number'] = '3+'
        sm_classes['Rewetting']['Summer Moisture Class Name'] = 'Moist'
        sm_classes['Rewetting']['Summer Moisture Class'] = '3+ (Moist)'
    elif user_input['Rewetting']['Median Groundwater Level in Summer'] >= -85 and user_input['Rewetting']['Median Groundwater Level in Summer'] < -45:
        sm_classes['Rewetting']['Summer Moisture Class Number'] = '2+'
        sm_classes['Rewetting']['Summer Moisture Class Name'] = 'Moderate moist'
        sm_classes['Rewetting']['Summer Moisture Class'] = '2+ (Moderate moist)'
    elif user_input['Rewetting']['Median Groundwater Level in Summer'] < -85:
        sm_classes['Rewetting']['Summer Moisture Class Number'] = '2-'
        sm_classes['Rewetting']['Summer Moisture Class Name'] = 'Moderate dry'
        sm_classes['Rewetting']['Summer Moisture Class'] = '2- (Moderate dry)'

    #Save Soil Moisture Classes tab as a pandas dataframe
    return pd.DataFrame.from_dict(sm_classes)

In [812]:
sm_classes = Create_Soil_Moisture_Classes_Tab('user_input_SET.json')
sm_classes

,Baseline,Rewetting
Summer Moisture Class Number,5+,2+
Summer Moisture Class Name,Wet (upper eulittoral),Moderate moist
Summer Moisture Class,5+ (Wet (upper eulittoral)),2+ (Moderate moist)


Functions corresponding to calculations used in the N2O fertilizer tab

In [813]:
#Direct N2O emissions

def Manure_CO2Site_Base(manure_applied_base, tot_area):
    EF=0.02
    N2O_CO2eq=265 #value from the CO2 equivalent table, just this value is used from the table
    N2O_N=EF*manure_applied_base
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_manure_base=N2O_site*N2O_CO2eq
    
    return CO2_site_manure_base

def Organic_Fert_CO2Site_Base(organic_applied_base,tot_area):
    EF=0.02
    N2O_CO2eq=265 
    N2O_N=EF*organic_applied_base
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_organic_base=N2O_site*N2O_CO2eq
    
    return CO2_site_organic_base

def Animal_Input_Base(animal):
    if animal == "dairy_cattle":
            EF_animal_b = 0.2
            n_excretion_value_b = 130.1
    elif animal == "beef_cattle":
            EF_animal_b = 0.2
            n_excretion_value_b = 28.2
    elif animal == "sheep":
            EF_animal_b = 0.1
            n_excretion_value_b = 9.9
    elif animal == "goat":
            EF_animal_b = 0.1
            n_excretion_value_b = 7.4
    elif animal == "buffalo":
            EF_animal_b = 0.2
            n_excretion_value_b = 76.5
    else:
        EF_animal_b = 0.0  
        n_excretion_value_b = 0.0  

    return EF_animal_b, n_excretion_value_b

def Grazing_CO2Site_Base(avg_n_animals_base, avg_days_base, n_excretion_value_b, EF_animal_b, tot_area):
    N2O_CO2eq=265
    N_amount_base=(n_excretion_value_b*avg_n_animals_base*(avg_days_base/365)) / tot_area
    N2O_N=EF_animal_b*N_amount_base
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_grazing_base=N2O_site*N2O_CO2eq
    
    return CO2_site_grazing_base, N_amount_base

def Synthetic_Fert_Input_Base(fertilizer):
    
    if fertilizer=="Nitrate Based":
        EF_fert_b=0.015
    elif fertilizer=="Ammonium Based":
        EF_fert_b==0.01
    else:
        EF_fert_b==0
        
    return EF_fert_b

def Synth_CO2Site_Base(fert_applied_base, EF_fert_b, tot_area): 
    N2O_CO2eq=265 
    N2O_N=EF_fert_b*fert_applied_base
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_synth_fert_base=N2O_site*N2O_CO2eq
    
    return CO2_site_synth_fert_base

def Residue_Left_Input_Base(answer_b, crop_name_b):
    crop_names = ["Cattail (Typha sp.)", "Reed (Pharagmites australis)", "Peat moss (Sphagnum sp.)", "Grasses like reed canary grass", "Alder (Alnus sp.)", "Other" ]  # List of all crop names
    
    if answer_b == "yes" and crop_name_b in crop_names:
        
        if crop_name_b == "Cattail (Typha sp.)":
            cropresidue_fraction_tot_yield_b = 0.116
        elif crop_name_b == "Reed (Pharagmites australis)":
            cropresidue_fraction_tot_yield_b = 0.0465
        else:
            cropresidue_fraction_tot_yield_b = 0.0
            
    else:
        cropresidue_fraction_tot_yield_b = 0.0
    
    return cropresidue_fraction_tot_yield_b

def Crop_Residue_Base(cropresidue_fraction_tot_yield_b, crop_yield_base, tot_area):
    nitrogen_content=0.015
    EF=0.02
    N2O_CO2eq=265
    amount_applied=cropresidue_fraction_tot_yield_b*crop_yield_base*1000*nitrogen_content
    N2O_N=EF*amount_applied
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_cropres_base=N2O_site*N2O_CO2eq
    
    return CO2_site_cropres_base

def Basis_value_managed_soils(tot_area):
    N2O_CO2eq=265
    N2O_ha=8 #Not sure why this value is 8
    N2O_site=N2O_ha*tot_area
    CO2_site_managed_soil=N2O_site*N2O_CO2eq
    
    return  CO2_site_managed_soil

def Total_Direct_N2Oemissions_Base(CO2_site_managed_soil, CO2_site_cropres_base, CO2_site_synth_fert_base, CO2_site_grazing_base, CO2_site_organic_base, CO2_site_manure_base ):
    total_direct_N2Oemiss_base= (CO2_site_managed_soil + CO2_site_cropres_base + CO2_site_synth_fert_base + CO2_site_grazing_base + CO2_site_organic_base + CO2_site_manure_base)/1000
    
    return total_direct_N2Oemiss_base

In [814]:
#Indirect N2O Emissions

def Animal_Ammonia_Base(manure_applied_base, N_amount_base, tot_area):
    EF_ammonia=0.104
    EF_N2O_N=0.01
    N2O_CO2eq=265
    ammonia_applied= N_amount_base*manure_applied_base
    volatilization_N= ammonia_applied*EF_ammonia
    N2O_N=EF_N2O_N*volatilization_N
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_animal_amm_base=N2O_site*N2O_CO2eq
    
    return CO2_site_animal_amm_base

def Fert_Ammonia_Base(fert_applied_base, EF_ammonia_b, tot_area):
    EF_N2O_N=0.01
    N2O_CO2eq=265
    volatilization_N=fert_applied_base* EF_ammonia_b
    N2O_N=EF_N2O_N*volatilization_N
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_fert_amm_base=N2O_site*N2O_CO2eq
    
    return CO2_site_fert_amm_base

def N_Oxide_Base(manure_applied_base, fert_applied_base, tot_area):
    EF_NOxide_ammonia=0.15
    EF_N2O_N=0.01
    N2O_CO2eq=265
    N=manure_applied_base + fert_applied_base
    volatilization_N= EF_NOxide_ammonia*N
    N2O_N=EF_N2O_N*volatilization_N
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_NOxide_base=N2O_site*N2O_CO2eq
    
    return CO2_site_NOxide_base

def Nitrate_Base(manure_applied_base, organic_applied_base, cropresidue_fraction_tot_yield_b, fert_applied_base, tot_area):
    EF_nitrate_leaching=0.3
    EF_N2O_N_leached=0.03
    N2O_CO2eq=265
    applied_animal=manure_applied_base + organic_applied_base
    applied_remaining=organic_applied_base + cropresidue_fraction_tot_yield_b
    nitrate_leached= (applied_animal +  fert_applied_base +  applied_remaining) * EF_nitrate_leaching
    N2O_N_leached= nitrate_leached * EF_N2O_N_leached
    N2O_ha= N2O_N_leached*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_nitrate_base=N2O_site*N2O_CO2eq
    
    return CO2_site_nitrate_base

def Total_Indirect_N2Oemissions_Base( CO2_site_nitrate_base, CO2_site_NOxide_base, CO2_site_fert_amm_base, CO2_site_animal_amm_base):
    tot_indirect_N2Oemiss_base=CO2_site_nitrate_base + CO2_site_NOxide_base + CO2_site_fert_amm_base + CO2_site_animal_amm_base
    return tot_indirect_N2Oemiss_base

Create a function for the construction of the Outcome tab using information from the user input json and the N2O, Crop Use, Data, and C Content Soil tabs, as well as the GEST 2.0 csv.

In [815]:
def Create_Outcome_Tab(user_input, data, crop_use, c_content, gest):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialize the Soil Moisture Classes dictionary
    outcome = {'Baseline': {}, 'Rewetting': {}, 'Creditable Year': {}}

    #Populate the Baseline dictionary
    outcome['Baseline']['Vegetation CH4 GWP'] = data.loc['Vegetation CH4 GWP']['Baseline']*user_input['General Site Data']['Total Area']
    outcome['Baseline']['Vegetation CO2 GWP'] = data.loc['Vegetation CO2 GWP']['Baseline']*user_input['General Site Data']['Total Area']

    outcome['Baseline']['Total Direct N2O'] = Total_Direct_N2Oemissions_Base(Basis_value_managed_soils(user_input['General Site Data']['Total Area']), 
                                                                            Crop_Residue_Base(Residue_Left_Input_Base(user_input['Baseline']['Crop Residue'], user_input['Baseline']['Crop']), user_input['Baseline']['Crop Yield'], user_input['General Site Data']['Total Area']), 
                                                                            Synth_CO2Site_Base(user_input['Baseline']['Amount Applied'], Synthetic_Fert_Input_Base(user_input['Baseline']['Synthetic Fertilizer']), user_input['General Site Data']['Total Area']),
                                                                            Grazing_CO2Site_Base(user_input['Baseline']['Average Number of Grazing Animals'], user_input['Baseline']['Average Number of Grazing Days per Year'], Animal_Input_Base(user_input['Baseline']['Grazing Animals'])[1], Animal_Input_Base(user_input['Baseline']['Grazing Animals'])[0], user_input['General Site Data']['Total Area'])[0],
                                                                            Organic_Fert_CO2Site_Base(user_input['Baseline']['Organic Fertilizer Applied'], user_input['General Site Data']['Total Area']),
                                                                            Manure_CO2Site_Base(user_input['Baseline']['Organic Fertilizer Applied'], user_input['General Site Data']['Total Area']))

    outcome['Baseline']['Total Indirect N2O'] = Total_Indirect_N2Oemissions_Base(Nitrate_Base(user_input['Baseline']['Animal Manure Applied'], user_input['Baseline']['Organic Fertilizer Applied'], Residue_Left_Input_Base(user_input['Baseline']['Crop Residue'], user_input['Baseline']['Crop']), user_input['Baseline']['Amount Applied'], user_input['General Site Data']['Total Area']), 
                                                                                 N_Oxide_Base(user_input['Baseline']['Animal Manure Applied'], user_input['Baseline']['Amount Applied'], user_input['General Site Data']['Total Area']), 
                                                                                 Fert_Ammonia_Base(user_input['Baseline']['Amount Applied'], Synthetic_Fert_Input_Base(user_input['Baseline']['Synthetic Fertilizer']), user_input['General Site Data']['Total Area']), 
                                                                                 Animal_Ammonia_Base(user_input['Baseline']['Animal Manure Applied'], Grazing_CO2Site_Base(user_input['Baseline']['Average Number of Grazing Animals'], user_input['Baseline']['Average Number of Grazing Days per Year'], Animal_Input_Base(user_input['Baseline']['Grazing Animals'])[1], Animal_Input_Base(user_input['Baseline']['Grazing Animals'])[0], user_input['General Site Data']['Total Area'])[1], user_input['General Site Data']['Total Area']))
    
    outcome['Baseline']['Activity'] = ((user_input['Baseline']['Litres of Diesel per Site']*3.35)+(user_input['Baseline']['Electricity per Site']*0.581))/1000
    outcome['Baseline']['Total'] = sum((float(outcome['Baseline']['Vegetation CH4 GWP']), float(outcome['Baseline']['Vegetation CO2 GWP']), float(outcome['Baseline']['Total Direct N2O']), float(outcome['Baseline']['Total Indirect N2O']), float(outcome['Baseline']['Activity'])), 0)
    
    #Populate the Rewetting dictionary
    outcome['Rewetting']['Vegetation CH4 GWP'] = data.loc['Vegetation CH4 GWP']['Rewetting']*user_input['General Site Data']['Total Area']
    outcome['Rewetting']['Vegetation CO2 GWP'] = data.loc['Vegetation CO2 GWP']['Rewetting']*user_input['General Site Data']['Total Area']

    outcome['Rewetting']['Total Direct N2O'] = Total_Direct_N2Oemissions_Base(Basis_value_managed_soils(user_input['General Site Data']['Total Area']), 
                                                                            Crop_Residue_Base(Residue_Left_Input_Base(user_input['Rewetting']['Crop Residue'], user_input['Rewetting']['Crop']), user_input['Rewetting']['Crop Yield'], user_input['General Site Data']['Total Area']), 
                                                                            Synth_CO2Site_Base(user_input['Rewetting']['Amount Applied'], Synthetic_Fert_Input_Base(user_input['Rewetting']['Synthetic Fertilizer']), user_input['General Site Data']['Total Area']),
                                                                            Grazing_CO2Site_Base(user_input['Rewetting']['Average Number of Grazing Animals'], user_input['Rewetting']['Average Number of Grazing Days per Year'], Animal_Input_Base(user_input['Rewetting']['Grazing Animals'])[1], Animal_Input_Base(user_input['Rewetting']['Grazing Animals'])[0], user_input['General Site Data']['Total Area'])[0],
                                                                            Organic_Fert_CO2Site_Base(user_input['Rewetting']['Organic Fertilizer Applied'], user_input['General Site Data']['Total Area']),
                                                                            Manure_CO2Site_Base(user_input['Rewetting']['Organic Fertilizer Applied'], user_input['General Site Data']['Total Area']))
    
    outcome['Rewetting']['Total Indirect N2O'] = Total_Indirect_N2Oemissions_Base(Nitrate_Base(user_input['Rewetting']['Animal Manure Applied'], user_input['Rewetting']['Organic Fertilizer Applied'], Residue_Left_Input_Base(user_input['Rewetting']['Crop Residue'], user_input['Rewetting']['Crop']), user_input['Rewetting']['Amount Applied'], user_input['General Site Data']['Total Area']), 
                                                                                 N_Oxide_Base(user_input['Rewetting']['Animal Manure Applied'], user_input['Rewetting']['Amount Applied'], user_input['General Site Data']['Total Area']), 
                                                                                 Fert_Ammonia_Base(user_input['Rewetting']['Amount Applied'], Synthetic_Fert_Input_Base(user_input['Rewetting']['Synthetic Fertilizer']), user_input['General Site Data']['Total Area']), 
                                                                                 Animal_Ammonia_Base(user_input['Rewetting']['Animal Manure Applied'], Grazing_CO2Site_Base(user_input['Rewetting']['Average Number of Grazing Animals'], user_input['Rewetting']['Average Number of Grazing Days per Year'], Animal_Input_Base(user_input['Rewetting']['Grazing Animals'])[1], Animal_Input_Base(user_input['Rewetting']['Grazing Animals'])[0], user_input['General Site Data']['Total Area'])[1], user_input['General Site Data']['Total Area']))
    
    outcome['Rewetting']['Product Ton CO2 per Site'] = crop_use.loc['Ton CO2 per Site']['Values']
    outcome['Rewetting']['Activity'] = ((user_input['Rewetting']['Litres of Diesel per Site']*3.35)+(user_input['Rewetting']['Electricity per Site']*0.581))/1000
    outcome['Rewetting']['Total'] = sum((float(outcome['Rewetting']['Vegetation CH4 GWP']), float(outcome['Rewetting']['Vegetation CO2 GWP']), float(outcome['Rewetting']['Total Direct N2O']), float(outcome['Rewetting']['Total Indirect N2O']), float(outcome['Rewetting']['Activity']), float(outcome['Rewetting']['Product Ton CO2 per Site'])), 0)

    #Populate Creditable Year tab
    veg_num_base = data['Baseline']['Vegetation Class'].split(':', 1)[0]
    veg_num_rewet = data['Rewetting']['Vegetation Class'].split(':', 1)[0]
    
    for i in range(2, 44):
        if veg_num_base == gest.iloc[i]['Name']:
            outcome['Creditable Year']['Baseline Scenario'] = float(c_content.loc['C-stock (Ton C per Ha)']['Values'])/(float(gest.iloc[i]['Total C-flux (ton C/ha)']))
        
        if veg_num_rewet == gest.iloc[i]['Name']:
            if float(c_content.loc['C-stock (Ton C per Ha)']['Values'])/float(gest.iloc[i]['Total C-flux (ton C/ha)']) > 0:
                outcome['Creditable Year']['Rewetting Scenario'] = float(c_content.loc['C-stock (Ton C per Ha)']['Values'])/float(gest.iloc[i]['Total C-flux (ton C/ha)'])
            else:
                outcome['Creditable Year']['Rewetting Scenario'] = float('inf')
    
    #Save the outcome tab in a pandas database
    return pd.DataFrame.from_dict(outcome)


In [816]:
outcome = Create_Outcome_Tab('user_input_SET.json', data_tab, crop_use_tab, c_content_tab, gest)
outcome

,Baseline,Rewetting,Creditable Year
Vegetation CH4 GWP,2.93,0.01,NaN
Vegetation CO2 GWP,-3.89,19.37,NaN
Total Direct N2O,2.4115,3.265179,NaN
Total Indirect N2O,819.531429,1033.783929,NaN
Activity,0.0,0.03931,NaN
Total,820.982929,1014.668417,NaN
Product Ton CO2 per Site,NaN,-41.8,NaN
Rewetting Scenario,NaN,NaN,66.174180
Baseline Scenario,NaN,NaN,-359.297774


Create Timeline tab using information from the user input json, GEST 2.0 csv, and the outcome and C content soil tabs.

In [817]:
def Create_Timeline_tab(user_input, outcome, c_content, gest):
        
    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)
        
    #Initialize the Timeline dict
    timeline = {}
    list_colnames = ['Baseline Emissions', 'Baseline_GESTv2', 'Rewetting Emissions', 'Rewetting_GESTv2', 'Carbon savings flow', 'Carbon savings stock', 'Carbon savings product', 'Carbon savings_tot', 'baseline GESTnr', 'C-sequestration baseline', 'C-stock soil (baseline)', 'Stock savings creditable?', 'Rewet GESTnr', 'C-sequestration rewet', 'C-credits_' + user_input['General Site Data']['Site Name']]
    for i in list_colnames:
        timeline[i] = {}
        for j in range(int(user_input['General Site Data']['Year Rewetting Started']), int(user_input['General Site Data']['Year Rewetting Started'])+50):
            timeline[i][j] = None

    #Populate dict row by row
    for i in range(list(timeline['Baseline Emissions'].keys())[0], list(timeline['Baseline Emissions'].keys())[-1]+1):
        timeline['Baseline Emissions'][i] = outcome.loc['Total']['Baseline']
        timeline['Baseline_GESTv2'][i] = user_input['Baseline']['Vegetation Class']
        timeline['Rewetting Emissions'][i] = outcome.loc['Total']['Rewetting']
        timeline['Rewetting_GESTv2'][i] = user_input['Rewetting']['Vegetation Class']
        timeline['Carbon savings flow'][i] = float(outcome.loc['Total Direct N2O']['Baseline']) + float(outcome.loc['Total Indirect N2O']['Baseline']) + float(outcome.loc['Activity']['Baseline']) - float(outcome.loc['Total Direct N2O']['Rewetting']) - float(outcome.loc['Total Indirect N2O']['Rewetting']) - float(outcome.loc['Activity']['Rewetting'])
        timeline['Carbon savings stock'][i] = float(outcome.loc['Vegetation CH4 GWP']['Baseline']) + float(outcome.loc['Vegetation CO2 GWP']['Baseline']) - float(outcome.loc['Vegetation CH4 GWP']['Rewetting']) - float(outcome.loc['Vegetation CO2 GWP']['Rewetting'])
        timeline['Carbon savings product'][i] = (-1)*float(outcome.loc['Product Ton CO2 per Site']['Rewetting'])
        timeline['Carbon savings_tot'][i] = sum((timeline['Carbon savings flow'][i], timeline['Carbon savings stock'][i], timeline['Carbon savings product'][i]), 0)
        timeline['baseline GESTnr'][i] = timeline['Baseline_GESTv2'][i].split(':', 1)[0]
        timeline['Rewet GESTnr'][i] = timeline['Rewetting_GESTv2'][i].split(':', 1)[0]

        for j in range(2, 44):
            if timeline['baseline GESTnr'][i] == gest.iloc[j]['Name']:
                timeline['C-sequestration baseline'][i] = float(gest.iloc[j]['Total C-flux (ton C/ha)'])*user_input['General Site Data']['Total Area']*(-1)
            if timeline['Rewet GESTnr'][i] == gest.iloc[j]['Name']:
                timeline['C-sequestration rewet'][i] = float(gest.iloc[j]['Total C-flux (ton C/ha)'])*user_input['General Site Data']['Total Area']*(-1)

        if i == list(timeline['Baseline Emissions'].keys())[0]:
            timeline['C-stock soil (baseline)'][i] = (c_content.loc['C-stock (Ton C per Ha)']['Values']*user_input['General Site Data']['Total Area']) + timeline['C-sequestration baseline'][i]
        else:
            timeline['C-stock soil (baseline)'][i] = timeline['C-stock soil (baseline)'][i-1] + timeline['C-sequestration baseline'][i]

        if timeline['C-stock soil (baseline)'][i] > 0:
            timeline['Stock savings creditable?'][i] = 1
        else:
            timeline['Stock savings creditable?'][i] = 0

        if timeline['Stock savings creditable?'][i] == 1:
            timeline['C-credits_' + user_input['General Site Data']['Site Name']][i] = timeline['Carbon savings_tot'][i]
        else:
            if timeline['C-sequestration rewet'][i] > 0:
                timeline['C-credits_' + user_input['General Site Data']['Site Name']][i] = timeline['Carbon savings flow'][i] + timeline['Carbon savings product'][i] + (timeline['C-sequestration rewet'][i]*(44/12))
            else:
                timeline['C-credits_' + user_input['General Site Data']['Site Name']][i] = timeline['Carbon savings flow'][i] + timeline['Carbon savings product'][i]
            
        

    #Save the outcome tab in a pandas database
    return pd.DataFrame.from_dict(timeline)
    

In [818]:
timeline = Create_Timeline_tab('user_input_SET.json', outcome, c_content_tab, gest)

Create Output tab using information from the user input json, C content soil, soil moisture classes, data, outcome, and crop use tabs.

In [819]:
def Create_Output_tab(user_input, sm_classes, data_tab, outcome, c_content):
    
    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialise Output dict
    output = {'Site Characteristics': {}, 'Baseline Outcomes': {}, 'Rewetting Outcomes': {}, 'Carbon Savings': {}}

    #Populate the Site Characteristics section
    output['Site Characteristics']['Site Name'] = user_input['General Site Data']['Site Name']
    output['Site Characteristics']['Coordinates'] = user_input['General Site Data']['Coordinates']
    output['Site Characteristics']['Total Area'] = user_input['General Site Data']['Total Area']
    output['Site Characteristics']['Peat Type'] = user_input['General Site Data']['Peat Type']
    output['Site Characteristics']['Peat Thickness'] = user_input['General Site Data']['Peat Thickness']
    output['Site Characteristics']['Year Rewetting Started'] = user_input['General Site Data']['Year Rewetting Started']
    output['Site Characteristics']['Baseline Soil Moisture Class'] = sm_classes.loc['Summer Moisture Class']['Baseline']
    output['Site Characteristics']['Rewetting Soil Moisture Class'] = sm_classes.loc['Summer Moisture Class']['Rewetting']
    output['Site Characteristics']['Baseline Vegetation Class'] = user_input['Baseline']['Vegetation Class']
    output['Site Characteristics']['Rewetting Vegetation Class'] = user_input['Rewetting']['Vegetation Class']

    #Populate the baseline outcomes section
    output['Baseline Outcomes']['CH4'] = float(data_tab.loc['Vegetation CH4 GWP']['Baseline']*user_input['General Site Data']['Total Area'])
    output['Baseline Outcomes']['CO2'] = float(data_tab.loc['Vegetation CO2 GWP']['Baseline']*user_input['General Site Data']['Total Area'])
    output['Baseline Outcomes']['C-Emission GWP Subtotal'] = float(output['Baseline Outcomes']['CH4']) + float(output['Baseline Outcomes']['CO2'])

    output['Baseline Outcomes']['N2O Direct'] = outcome.loc['Total Direct N2O']['Baseline']
    output['Baseline Outcomes']['N2O Indirect'] = outcome.loc['Total Indirect N2O']['Baseline']
    output['Baseline Outcomes']['N2O-Emission GWP Subtotal'] = float(output['Baseline Outcomes']['N2O Direct']) + float(output['Baseline Outcomes']['N2O Indirect'])

    output['Baseline Outcomes']['Activity GWP Subtotal'] = outcome.loc['Activity']['Baseline']
    output['Baseline Outcomes']['GWP Total'] = outcome.loc['Total']['Baseline']

    #Populate the rewetting outcomes section
    output['Rewetting Outcomes']['CH4'] = float(data_tab.loc['Vegetation CH4 GWP']['Rewetting']*user_input['General Site Data']['Total Area'])
    output['Rewetting Outcomes']['CO2'] = float(data_tab.loc['Vegetation CO2 GWP']['Rewetting']*user_input['General Site Data']['Total Area'])
    output['Rewetting Outcomes']['C-Emission GWP Subtotal'] = float(output['Rewetting Outcomes']['CH4']) + float(output['Rewetting Outcomes']['CO2'])

    output['Rewetting Outcomes']['N2O Direct'] = outcome.loc['Total Direct N2O']['Rewetting']
    output['Rewetting Outcomes']['N2O Indirect'] = outcome.loc['Total Indirect N2O']['Rewetting']
    output['Rewetting Outcomes']['N2O-Emission GWP Subtotal'] = float(output['Rewetting Outcomes']['N2O Direct']) + float(output['Rewetting Outcomes']['N2O Indirect'])

    output['Rewetting Outcomes']['Activity GWP Subtotal'] = outcome.loc['Activity']['Rewetting']
    output['Rewetting Outcomes']['Product GWP Subtotal'] = outcome.loc['Product Ton CO2 per Site']['Rewetting']
    output['Rewetting Outcomes']['GWP Total'] = outcome.loc['Total']['Rewetting']

    #Populate Carbon Savings section
    output['Carbon Savings']['GHG Savings Total per Year per Site'] = output['Baseline Outcomes']['GWP Total'] - output['Rewetting Outcomes']['GWP Total']
    output['Carbon Savings']['GHG Savings Total per Year per Hectare'] = output['Carbon Savings']['GHG Savings Total per Year per Site']/output['Site Characteristics']['Total Area']
    output['Carbon Savings']['GHG Savings Stock per Year per Site'] = float(outcome.loc['Vegetation CH4 GWP']['Baseline']) + float(outcome.loc['Vegetation CO2 GWP']['Baseline']) - float(outcome.loc['Vegetation CH4 GWP']['Rewetting']) - float(outcome.loc['Vegetation CO2 GWP']['Rewetting'])
    output['Carbon Savings']['GHG Savings Stock per Year per Hectare'] = output['Carbon Savings']['GHG Savings Stock per Year per Site']/output['Site Characteristics']['Total Area']
    output['Carbon Savings']['GHG Savings Product Use per Year per Site'] = output['Rewetting Outcomes']['Activity GWP Subtotal']*(-1)
    output['Carbon Savings']['GHG Savings Product Use per Year per Hectare'] = output['Carbon Savings']['GHG Savings Product Use per Year per Site']/output['Site Characteristics']['Total Area']

    output['Carbon Savings']['Carbon Stock Peat Soil at Start Year (tCO2-eq/site)'] = c_content.loc['C-stock (tCO2-eq per Ha)']['Values']*output['Site Characteristics']['Total Area']
    output['Carbon Savings']['Carbon Stock Peat Soil at Start Year (ton C/site)'] = c_content.loc['C-stock (Ton C per Ha)']['Values']*output['Site Characteristics']['Total Area']
    output['Carbon Savings']['Carbon Stock Peat Soil at Start Year (tCO2-eq/ha)'] = c_content.loc['C-stock (tCO2-eq per Ha)']['Values']
    output['Carbon Savings']['Carbon Stock Peat Soil at Start Year (ton C/ha)'] = c_content.loc['C-stock (Ton C per Ha)']['Values']

    output['Carbon Savings']['Time until Peat is Lost (Baseline Scenario)'] = outcome.loc['Baseline Scenario']['Creditable Year']
    output['Carbon Savings']['Time until Peat is Lost (Rewetting Scenario)'] = outcome.loc['Rewetting Scenario']['Creditable Year']
    
    with open('output_SET.json', 'w') as outfile:
        print(json.dumps(output, indent = 5), file = outfile)
    

In [820]:
Create_Output_tab('user_input_SET.json', sm_classes, data_tab, outcome, c_content_tab)